# daruでのデータの検索と結合

## Arelライクなクエリシンタックス

[Arel](https://github.com/rails/arel)は、最も人気の高いRubyフレームワークであるRailsの主要コンポーネントの1つであり，非常に人気の高いgemです。
ArelはRubyのメソッドを連鎖させてSQL文字列を作成する美しく直感的な構文を公開するORMヘルパーです。
daruでは、この構文をうまく採用しました。
結果は、DataFrameまたはVectorから任意の種類のデータを取得するための非常に直感的で読みやすい構文です。
この構文がいくつかの例でどのように動作するかを見てみましょう：


In [11]:
require 'daru'

false

この構文を使用するには、`Daru::Vector`で定義されているコンパレータメソッドを呼び出し、その結果をDataFrameとVectorの両方で使用できる`#where`メソッドに渡します。
簡単な例を示します。


In [12]:
vector = Daru::Vector.new([2,4,5,51,5,16,2,5,3,2,1,5,2,5,2,1,56,234,6,21])
vector.where((vector.eq(5) | vector.eq(1)) & vector.mt(2))

#<Daru::Vector(5)>
   2   5
   4   5
   7   5
  11   5
  13   5

**論理和（||）または論理AND（&&）ではなく、論理和（|）と論理和（＆）を使用することに注意してください。**

このように `Vector#eq`メソッドによって返された結果は`Vector#`によって評価され、生成されたVectorが返されます。
インデックスも保持されます。

where句はDataFrameでも使用でき、同様の結果が得られます。

In [13]:
df = Daru::DataFrame.new({
  a: [1,2,3,4,5,6]*100,
  b: ['a','b','c','d','e','f']*100,
  c: [11,22,33,44,55,66]*100
}, index: (1..600).to_a.shuffle)

df.where(df[:a].eq(2) | df[:c].eq(55))

#<Daru::DataFrame(200x3)>
       a   b   c
  17   2   b  22
 338   5   e  55
  63   2   b  22
 267   5   e  55
 120   2   b  22
 595   5   e  55
 381   2   b  22
 518   5   e  55
 486   2   b  22
 318   5   e  55
 217   2   b  22
 441   5   e  55
 157   2   b  22
 470   5   e  55
 290   2   b  22
 ... ... ... ...

Vectorのコンパレータメソッドは、`Daru::Core:: Query::BoolArray`型のオブジェクトを返します。
これにより、OR演算とAND演算を実行できます。 

読みやすくするために、`#&`および`#|`の代わりに`#and`または`#or`を使用することもできます。

In [14]:
vector.where(vector.eq(2).or(vector.eq(5)).and(vector.mt(2)))

#<Daru::Vector(5)>
   2   5
   4   5
   7   5
  11   5
  13   5

`DataFrame#filter`や`Vector#keep_if`のような他のより堅牢なメソッドよりもwhere節を使用する主な利点の1つは、（破壊的ではありませんが）はるかに高速であるということです。
[これらのベンチマーク](https://github.com/v0dro/daru/blob/master/benchmarks/where_vs_filter.rb)は私の主張を証明しています。


## データフレーム間の結合の実行

`Daru::DataFrame`は、2つのDataFrames間でSQLスタイルの結合を実行するための`#join`メソッドを提供します。
現在`#join`は、DataFrames間の内側、左外部、右外部、および完全外部結合をサポートしています。

実演するには下記になります。

In [15]:
left = Daru::DataFrame.new({
  :id   => [1,2,3,4],
  :name => ['Pirate', 'Monkey', 'Ninja', 'Spaghetti']
})
right = Daru::DataFrame.new({
  :id => [1,2,3,4],
  :name => ['Rutabaga', 'Pirate', 'Darth Vader', 'Ninja']
})
nil

`:name`列の内部結合を実行するには下記になります。


In [16]:
left.join(right, on: [:name], how: :inner)

#<Daru::DataFrame(2x3)>
          id_1   name   id_2
      0      3  Ninja      4
      1      1 Pirate      2

外部左結合は、以下で可能です。

In [17]:
left.join(right, on: [:name], how: :left)

#<Daru::DataFrame(4x3)>
                id_1      name      id_2
         0         2    Monkey       nil
         1         3     Ninja         4
         2         1    Pirate         2
         3         4 Spaghetti       nil

外部右結合は、以下で可能です。

In [18]:
left.join(right, on: [:name], how: :right)

#<Daru::DataFrame(4x3)>
                  id_1       name       id_2
          0        nil Darth Vade          3
          1          3      Ninja          4
          2          1     Pirate          2
          3        nil   Rutabaga          1

最後に、完全外部結合になります。

In [19]:
left.join(right, on: [:name], how: :outer)

#<Daru::DataFrame(6x3)>
                  id_1       name       id_2
          0        nil Darth Vade          3
          1          2     Monkey        nil
          2          3      Ninja          4
          3          1     Pirate          2
          4        nil   Rutabaga          1
          5          4  Spaghetti        nil